# Cars vs Bikes Image Classification (Neural Network)

This notebook trains a simple Neural Network using gradient descent to classify Cars vs Bikes images.

In [ ]:

from google.colab import files
uploaded = files.upload()


In [ ]:

import zipfile, os

zip_path = list(uploaded.keys())[0]
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted to", extract_path)


In [ ]:

import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [ ]:

IMG_SIZE = 64
DATASET_PATH = "/content/dataset"

X, Y = [], []

for label, folder in enumerate(["Cars", "Bikes"]):
    folder_path = os.path.join(DATASET_PATH, folder)
    for img_name in os.listdir(folder_path):
        img = Image.open(os.path.join(folder_path, img_name)).convert("RGB")
        img = img.resize((IMG_SIZE, IMG_SIZE))
        X.append(np.array(img).flatten() / 255.0)
        Y.append(label)

X = np.array(X).T
Y = np.array(Y).reshape(1, -1)

X_train, X_test, Y_train, Y_test = train_test_split(
    X.T, Y.T, test_size=0.2, random_state=42
)

X_train, X_test = X_train.T, X_test.T
Y_train, Y_test = Y_train.T, Y_test.T


In [ ]:

def sigmoid(z):
    return 1/(1+np.exp(-z))

def relu(z):
    return np.maximum(0,z)


In [ ]:

def initialize_parameters(n_x, n_h):
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(1, n_h) * 0.01
    b2 = np.zeros((1,1))
    return W1,b1,W2,b2


In [ ]:

def forward(X,W1,b1,W2,b2):
    Z1 = np.dot(W1,X)+b1
    A1 = relu(Z1)
    Z2 = np.dot(W2,A1)+b2
    A2 = sigmoid(Z2)
    return A1,A2


In [ ]:

def cost(A2,Y):
    m = Y.shape[1]
    return -(1/m)*np.sum(Y*np.log(A2+1e-8)+(1-Y)*np.log(1-A2+1e-8))


In [ ]:

def accuracy(X,Y,W1,b1,W2,b2):
    _,A2 = forward(X,W1,b1,W2,b2)
    return np.mean((A2>0.5)==Y)*100


In [ ]:

n_x = X_train.shape[0]
n_h = 64
lr = 0.01
iters = 1000

W1,b1,W2,b2 = initialize_parameters(n_x,n_h)
costs = []

for i in range(iters):
    A1,A2 = forward(X_train,W1,b1,W2,b2)
    c = cost(A2,Y_train)

    dZ2 = A2 - Y_train
    dW2 = np.dot(dZ2,A1.T)/X_train.shape[1]
    db2 = np.sum(dZ2,axis=1,keepdims=True)/X_train.shape[1]

    dA1 = np.dot(W2.T,dZ2)
    dZ1 = dA1*(A1>0)
    dW1 = np.dot(dZ1,X_train.T)/X_train.shape[1]
    db1 = np.sum(dZ1,axis=1,keepdims=True)/X_train.shape[1]

    W1 -= lr*dW1
    b1 -= lr*db1
    W2 -= lr*dW2
    b2 -= lr*db2

    if i%100==0:
        costs.append(c)


In [ ]:

plt.plot(costs)
plt.xlabel("Iterations (x100)")
plt.ylabel("Cost")
plt.title("Training Cost Curve")
plt.show()

print("Training Accuracy:", accuracy(X_train,Y_train,W1,b1,W2,b2))
print("Testing Accuracy:", accuracy(X_test,Y_test,W1,b1,W2,b2))
